In [1]:
import random
import numpy as np
import igraph
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn import preprocessing
import nltk
import csv

with open("testing_set.txt", "r") as f:
    reader = csv.reader(f)
    testing_set  = list(reader)

testing_set = [element[0].split(" ") for element in testing_set]

###################
# random baseline #
###################

random_predictions = np.random.choice([0, 1], size=len(testing_set))
random_predictions = zip(range(len(testing_set)),random_predictions)

def write(prediction, filename) :
    with open(filename,"w") as pred:
        csv_out = csv.writer(pred)
        csv_out.writerow(('id', 'category'))
        for row in prediction:
            csv_out.writerow(row)

## Chargement des fichiers et séparation train_test_split

In [2]:
with open("training_set.txt", "r") as f:
    reader = csv.reader(f)
    training_set  = list(reader)

training_set = [element[0].split(" ") for element in training_set]
y = [elt[-1] for elt in training_set]
y = np.array(y).astype(int)
with open("node_information.csv", "r") as f:
    reader = csv.reader(f)
    node_info  = list(reader)

IDs = [element[0] for element in node_info]

In [3]:
## idée rajouter titre à l'abstract
corpus = [element[5] + ' ' + element[2].lower() for element in node_info]

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import nltk
#nltk.download('stopwords'); nltk.download('punkt'); nltk.download('wordnet')
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
def tokenize(text) :
    tokens = word_tokenize(text)
    result = [stemmer.stem(i) for i in tokens if((i.isalpha()) & (i not in stop_words)) ]
    return ' '.join(result)
## paralellisation de la lemmatisation des textes
from multiprocessing import Pool
with Pool(8) as p: 
    corpus_new = p.map(tokenize, corpus)

## création d'un dataframe pour stocker les informations des noeuds
dico_node_info = {elt[0]: {'pub_year': int(elt[1]), 'authors' : elt[3].split(','),
                                'abstract' : corpus_new[i].split()} for i, elt in enumerate(node_info)}

### Suppression volontaire de quelques liens pour valider le modèle

In [4]:
import pandas as pd
import warnings
warnings.simplefilter('ignore')
data_ = pd.DataFrame({'node_1':[int(elt[0]) for elt in training_set], 
                     'node_2':[int(elt[1]) for elt in training_set],
                    'link' : [int(elt[2]) for elt in training_set]})
#fb_df_temp = data_[data_.link == 1]
#fb_df_temp.link = 1 - fb_df_temp.link
#fb_df_temp.columns = ['node_2', 'node_1', 'link']

#data_ = data_.append(fb_df_temp)
import networkx as nx
import tqdm
fb_df_temp = data_[data_.link == 1]
initial_node_count = len(set(fb_df_temp.node_1.unique()).union(set(fb_df_temp.node_2.unique())))
# empty list to store removable links
omissible_links_index = []
for i in tqdm.tqdm(data_[data_.link == 1].index.values):
    break
    # remove a node pair and build a new graph
    tmp = fb_df_temp.drop(index = i) 
    all_nodes = set(tmp.node_1.unique()).union(set(tmp.node_2.unique()))
    if len(all_nodes) == initial_node_count :
        omissible_links_index.append(i)
        fb_df_temp = fb_df_temp.drop(index = i)        
    else :
        pass
    if len(omissible_links_index) == 30000 :
        break
#np.save('omissible.npy', omissible_links_index)
omissible_links_index = np.load('omissible.npy')
fb_df_partial = data_[data_.link == 1].drop(index = omissible_links_index).drop(columns=['link'])

  0%|          | 0/335130 [00:00<?, ?it/s]


In [5]:
all_nodes = set(fb_df_temp.node_1.unique()).union(set(fb_df_temp.node_2.unique()))
nodes_bizz = [node for node in dico_node_info.keys() if int(node) not in all_nodes]

In [6]:
G_data = nx.from_pandas_edgelist(fb_df_partial, "node_1", "node_2", create_using=nx.DiGraph())
G_data.add_nodes_from(nodes_bizz)

In [6]:
len(G_data)

NameError: name 'G_data' is not defined

In [ ]:
all_pairs = [tuple(elt[:2]) for elt in np.array(training_set).astype(int)] +\
            [tuple(elt[:2]) for elt in np.array(testing_set).astype(int)]

## Extraction des features

### Working with attributes of nodes

In [14]:
tmp = pd.Series(''.join(corpus_new).split()).value_counts()

In [21]:
tmp[tmp > 2].shape

(8705,)

In [9]:
### TF-IDF pour la similarité entre abstract + titre
vectorizer = TfidfVectorizer(decode_error='ignore', smooth_idf=False, stop_words=stop_words,
                      encoding='utf-8', min_df = 2, max_df=5000)
# each row is a node in the order of node_info
corpus_idf = vectorizer.fit_transform(corpus_new)
corpus_idf.shape

(27770, 8868)

In [10]:
## Etude du delta entre les années
tmp_ = pd.DataFrame.from_dict(dico_node_info).T[['pub_year', 'authors']]
tmp_['node_1'] = tmp_.index.astype(int)
tmp = pd.merge(tmp_, data_, on='node_1')
tmp = tmp.rename(columns = {"pub_year":"pub_year_1"})
tmp_ = tmp_.rename(columns = {"node_1":"node_2"})
tmp = pd.merge(tmp_, tmp, on='node_2')
tmp = tmp.rename(columns = {"pub_year":"pub_year_2"})
tmp['diff_year'] = tmp.pub_year_1 - tmp.pub_year_2

In [11]:
## 98% des delta_year < 0 sont pas connectés 
100*( 1 - len(tmp[(tmp.diff_year < 0 ) & (tmp.link == 1)])/(tmp.diff_year < 0 ).sum() )

98.35963022172794

In [51]:
tmp[(tmp.diff_year < 0 ) & (tmp.link == 1)].head()

,pub_year_2,authors_x,node_2,pub_year_1,authors_y,node_1,link,diff_year
768,2000,"[Csaba Csaki, Joshua Erlich, Timothy J. Holl...",1033,1999,"[O. DeWolfe, D.Z. Freedman, S.S. Gubser, , ...",9909134,1,-1
1004,2000,"[O. Kenneth, S. Nussinov]",1045,1999,"[O. Kenneth, S. Nussinov]",9912291,1,-1
1307,2001,[Kenji Suzuki],1057,2000,[M. Caselle],3119,1,-1
1677,2002,[A. Sagnotti],1077,2000,"[Sunil Mukhi, Nemani V. Suryanarayana (Tata I...",3219,1,-2
1679,2002,[A. Sagnotti],1077,2000,"[C. Angelantonj, I. Antoniadis, E. Dudas, A...",7090,1,-2


In [55]:
get_graph_feature(('9912291', '1045'))

array([1.44045518e-04, 3.60113796e-05, 1.47016829e-05, 5.18727784e-09,
       9.06481076e-11, 3.12322908e-12, 2.00000000e+00, 6.66666667e-01,
       4.00000000e-01, 1.82047845e+00, 1.00000000e+00, 1.00000000e+00])

In [13]:
def get_features(tuple_to_process) : 
    ret = []
    source = tuple_to_process[0]
    target = tuple_to_process[1]
    
    source_info = dico_node_info[source]
    target_info = dico_node_info[target]
    
    source_abstract = set(source_info['abstract'])
    target_abstract = set(target_info['abstract'])
    
    source_auth = set(source_info['authors'])
    target_auth = set(target_info['authors'])
    
    ## delta entre dates de publication
    delta_year = int(source_info['pub_year']) - int(target_info['pub_year'])
    if delta_year < 0 :
        delta_year = 0
    else :
        delta_year = 1
    
    ## cosine similiarity et intersection auteurs abstracts
    common_words = len(source_abstract.intersection(target_abstract))/(len(source_abstract.union(target_abstract)))
    common_auths = len(source_auth.intersection(target_auth))/(len(source_auth.union(target_auth)))
    
    cosine_sim = corpus_idf[IDs.index(source)].dot( 
                        corpus_idf[IDs.index(target)].T).toarray()[0][0] ## les vecteurs sont déjà normés 
    
    ret.append(common_words)
    ret.append(cosine_sim)
    ret.append(delta_year)
    ret.append(common_auths)
    
    return np.array(ret)

In [85]:
#sim = corpus_idf[IDs.index(target)].dot(corpus_idf[IDs.index(source)].T).toarray()[0, 0]
#source, target = pairs_train[0]

### features liées à la structure du graphe

In [14]:
def common_neighbors(G, ebunch = None):
    u = ebunch[0]
    v = ebunch[1]
    if str(u) in nodes_bizz :
        u = str(u)
    if str(v) in nodes_bizz :
        v = str(v)
    common_neighbors =  [w for w in G[u] if w in G[v] and w not in (u, v)]
    return common_neighbors

In [15]:
dico_common_neighbors = {}
for elt in tqdm.tqdm(all_pairs) :
    dico_common_neighbors[elt] = common_neighbors(G_data, elt)

100%|██████████| 648160/648160 [00:25<00:00, 25922.85it/s]


In [16]:
def resource_allocation_index(G, ebunch=None):
    if ebunch is None:
        ebunch = nx.non_edges(G)
    def predict(u, v):
        key_ =  (u, v)
        return sum(1 / G.degree(w) for w in dico_common_neighbors.get(key_, []))
    return predict(ebunch[0], ebunch[1])
def jaccard_coefficient(G, ebunch=None):
    if ebunch is None:
        ebunch = nx.non_edges(G)
    def predict(u, v):
        key_ =  (u, v)
        cnbors = list(dico_common_neighbors.get(key_,[]))
        union_size = len(set(G[u]) | set(G[v]))
        if union_size == 0:
            return 0
        else:
            return len(cnbors) / union_size
    
    return predict(ebunch[0], ebunch[1])
import math
def adamic_adar_index(G, ebunch=None):
    if ebunch is None:
        ebunch = nx.non_edges(G)
  
    def predict(u, v):
        key_ =  (u, v)
        return sum(1 / math.log(G.degree(w)) for w in dico_common_neighbors.get(key_, []))
  
    return predict(ebunch[0], ebunch[1])
def resource_allocation_CNI(G, ebunch = None):
    u = ebunch[0]
    v = ebunch[1]
    key_ =  (u, v)
    common_neighbors = dico_common_neighbors.get(key_,[])
    first = sum([1/(len(set(G.neighbors(i)))) for i in common_neighbors if (len(set(G.neighbors(i)))) != 0])

    #neighbor_x = G.neighbors(u)
    #neighbor_y = G.neighbors(v)

    #second = sum([(1/(len(set(G.neighbors(cross[0]))))) - (1/(len(set(G.neighbors(cross[1]))))) for cross in list(itertools.product(list(neighbor_x), list(neighbor_y))) 
    #            if cross in list(G.edges)])

    return first

In [17]:
out_degree_centrality = nx.out_degree_centrality(G_data)
in_degree_centrality = nx.in_degree_centrality(G_data)
page_rank = nx.pagerank_scipy(G_data)
hub_score, authority_score = nx.hits(G_data)

In [75]:
import itertools
def get_graph_feature(tuple_to_process) :
    ret = []
    source = tuple_to_process[0]
    target = tuple_to_process[1]
    if source not in nodes_bizz :
        source = int(source)
    if target not in nodes_bizz :
        target = int(target)
    ebunch = (source, target)    
    ret.append(out_degree_centrality[source])
    ret.append(in_degree_centrality[target])
    ret.append(page_rank[target])
    ret.append(out_degree_centrality[source] * in_degree_centrality[target])
    ret.append(hub_score[source])
    ret.append(authority_score[target])
    ret.append(len(dico_common_neighbors.get(ebunch, [])))
    ret.append(resource_allocation_index(G_data, ebunch))
    ret.append(jaccard_coefficient(G_data, ebunch))
    ret.append(adamic_adar_index(G_data, ebunch))
    ret.append(resource_allocation_CNI(G_data, ebunch))
    #try :
    #    ret.append(nx.shortest_path_length(G_data, source, target))
    #except :
    #    ret.append(-1.)
    return np.array(ret)

In [19]:
pairs_train = [tuple(elt[0 : 2]) for i, elt in enumerate(training_set) if i not in omissible_links_index]
y_train = [int(elt[-1]) for i, elt in enumerate(training_set) if i not in omissible_links_index]
len(pairs_train)

585512

In [76]:
with Pool(6) as p: 
    features_nodes = p.map(get_features, pairs_train)
features_nodes = np.array(features_nodes)

with Pool(6) as p: 
    train_tr = p.map(get_graph_feature, pairs_train )
train_tr = np.asarray(train_tr)

In [21]:
print(features_nodes.shape)
print(train_tr.shape)

(585512, 4)
(585512, 12)


In [22]:
pairs_val = [tuple(elt[0 : 2]) for i, elt in enumerate(training_set) if i in omissible_links_index]
y_val = np.array([int(elt[-1]) for i, elt in enumerate(training_set) if i in omissible_links_index])
len(pairs_val)

30000

In [23]:
y_train = np.array(y_train)
y_val = np.array(y_val)

In [78]:
with Pool(4) as p: 
    features_val = p.map(get_features, pairs_val)
features_val = np.array(features_val)
#val_tr = []
with Pool(6) as p: 
    val_tr = p.map(get_graph_feature, pairs_val)
val_tr = np.asarray(val_tr)

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [26]:
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
import sklearn
def train_logistic(train, val, c_inf=-1, c_up=2, n=20) : 
    logistic = linear_model.LogisticRegression()
    C = np.logspace(c_inf, c_up, n)
    params = dict(C=C, class_weight=['balanced'], solver=['lbfgs'])
    clf = GridSearchCV(logistic, params, cv=3, verbose=10, n_jobs = -1,  scoring='f1')
    clf.fit(train, y_train)
    y_pred = clf.predict(val)
    print(sklearn.metrics.f1_score(y_val, y_pred))
    return clf

In [27]:
# initialize basic SVM
def train_svm(train, val) : 
    classifier = svm.LinearSVC()
    C = np.logspace(0, 1, 2)
    params = dict(C=C, class_weight=['balanced'])
    clf = GridSearchCV(classifier, params, cv=3, verbose=10, n_jobs = -1,  scoring='f1')
    clf.fit(train, y_train)
    y_pred = clf.predict(val)
    print(sklearn.metrics.f1_score(y_val, y_pred))
    return clf

In [28]:
## f-1 score
import tensorflow.keras.backend as K
import sklearn
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

### Avec uniquement les features graph

In [81]:
train = scaler.fit_transform(train_tr)
val = scaler.transform(val_tr)
print(train.shape, val.shape)

(585512, 11) (30000, 11)


In [82]:
clf = train_logistic(train, val)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   21.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   23.9s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   43.0s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   54.1s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  52 out of  60 | elapsed:  1.3min remaining:   11.6s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  1.4min finished


0.906904718527965


In [83]:
clf.best_params_

{'C': 1.2742749857031335, 'class_weight': 'balanced', 'solver': 'lbfgs'}

In [176]:
#train_svm(train, val)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  15 | elapsed:  2.6min remaining: 17.0min
[Parallel(n_jobs=-1)]: Done   4 out of  15 | elapsed:  2.7min remaining:  7.5min
[Parallel(n_jobs=-1)]: Done   6 out of  15 | elapsed:  2.7min remaining:  4.1min
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:  2.9min remaining:  2.5min
[Parallel(n_jobs=-1)]: Done  10 out of  15 | elapsed:  5.1min remaining:  2.5min
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:  5.1min remaining:  1.3min
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  5.3min finished


0.7223983405737544


### Avec uniquement les attributs

In [87]:
train = features_nodes #scaler.fit_transform(features_nodes)
val = features_val #scaler.transform(features_val)
print(train.shape, val.shape)

(585512, 4) (30000, 4)


In [88]:
clf = train_logistic(train, val, c_inf=-3, c_up=-1, n=30)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   14.6s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:   23.3s
[Parallel(n_jobs=-1)]: Done  85 out of  90 | elapsed:   28.7s remaining:    1.7s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:   29.7s finished


0.9218716308488464


In [89]:
clf.best_params_

{'C': 0.0041753189365604, 'class_weight': 'balanced', 'solver': 'lbfgs'}

In [204]:
#train_svm(train, val)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed:   57.2s remaining:  1.9min
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:  1.0min remaining:  1.0min
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:  1.2min remaining:   35.9s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  1.3min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  1.3min finished


0.835808495170838


### En concatenant les deux

In [90]:
train = np.hstack((train_tr, features_nodes))
val = np.hstack((val_tr, features_val))
train = scaler.fit_transform(train)
val = scaler.transform(val)
print(train.shape, val.shape)

(585512, 15) (30000, 15)


#### Regression logistic

In [91]:
clf = train_logistic(train, val)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   22.8s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   25.7s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   46.0s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   58.2s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  52 out of  60 | elapsed:  1.4min remaining:   12.8s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  1.5min finished


0.9585864546811539


In [92]:
clf.best_params_

{'C': 100.0, 'class_weight': 'balanced', 'solver': 'lbfgs'}

#### xgboost

In [256]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
params = {'subsample': 0.7,
 'reg_lambda': 1,
 'reg_alpha': 0.1,
 'n_estimators': 200,
 'min_samples_split': 3,
 'min_samples_leaf': 3,
 'min_child_weight': 7,
 'max_depth': 30,
 'learning_rate': 0.1,
 'gamma': 0.1,
 'criterion': 'entropy',
 'colsample_bytree': 1,
 'colsample_bylevel': 0.7,
         'n_jobs':-1}
clf = XGBClassifier()
clf.set_params(**params)
clf.fit(train, y_train)
y_pred = clf.predict(val)
print(sklearn.metrics.f1_score(y_val, y_pred))

0.9481434732302514


#### Neural network

In [95]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional, Input, BatchNormalization
from tensorflow.core.protobuf import rewriter_config_pb2
import tensorflow.keras.backend as K
tf.keras.backend.clear_session()  # For easy reset of notebook state.
model = Sequential()
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(5, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=[f1_m])
bs = 2048
n_epochs = 30
history = model.fit(train, np.array(y_train),
                    batch_size=bs, epochs=n_epochs,
                    validation_data=(val, np.array(y_val)))
y_pred = model.predict(val)
y_pred = np.where(y_pred > 0.5, 1, 0)
print(sklearn.metrics.f1_score(y_val, y_pred))

Train on 585512 samples, validate on 30000 samples
Epoch 1/30
585512/585512 [==============================] - 1s 2us/sample - loss: 0.2761 - f1_m: 0.8979 - val_loss: 0.1646 - val_f1_m: 0.9626
Epoch 2/30
585512/585512 [==============================] - 1s 1us/sample - loss: 0.1750 - f1_m: 0.9393 - val_loss: 0.1449 - val_f1_m: 0.9680
Epoch 3/30
585512/585512 [==============================] - 1s 1us/sample - loss: 0.1658 - f1_m: 0.9426 - val_loss: 0.1482 - val_f1_m: 0.9688
Epoch 4/30
585512/585512 [==============================] - 1s 1us/sample - loss: 0.1616 - f1_m: 0.9439 - val_loss: 0.1510 - val_f1_m: 0.9686
Epoch 5/30
585512/585512 [==============================] - 1s 1us/sample - loss: 0.1585 - f1_m: 0.9449 - val_loss: 0.1502 - val_f1_m: 0.9687
Epoch 6/30
585512/585512 [==============================] - 1s 1us/sample - loss: 0.1569 - f1_m: 0.9454 - val_loss: 0.1475 - val_f1_m: 0.9694
Epoch 7/30
585512/585512 [==============================] - 1s 1us/sample - loss: 0.1548 - f1_m: 

## Graph embedding

In [110]:
#!pip install node2vec

In [7]:
G_data = nx.from_pandas_edgelist(fb_df_partial, "node_1", "node_2", create_using=nx.Graph())
G_data.add_nodes_from(nodes_bizz)

from node2vec import Node2Vec
node2vec = Node2Vec(G_data, dimensions=128, walk_length=50, 
                    num_walks=30, workers=1, temp_folder='')
# train node2vec model
n2w_model = node2vec.fit(window=5, min_count=1)
n2w_model.wv.save_word2vec_format('node_emb_partiel')

Generating walks (CPU: 1): 100%|██████████| 30/30 [28:46<00:00, 57.55s/it]


In [ ]:
edges = data_[data_.link == 1]
G_data = nx.from_pandas_edgelist(edges, "node_1", "node_2", create_using=nx.Graph())
G_data.add_nodes_from(nodes_bizz)

from node2vec import Node2Vec
node2vec = Node2Vec(G_data, dimensions=128, walk_length=50, 
                    num_walks=50, workers=4, temp_folder='')
# train node2vec model
n2w_model = node2vec.fit(window=8, min_count=1)
n2w_model.wv.save_word2vec_format('node_emb_total_')

Computing transition probabilities:   1%|          | 139/27770 [00:03<10:18, 44.70it/s]

### Node embeddings + NN

In [396]:
from node2vec import Node2Vec
node2vec = Node2Vec(G_data, dimensions=128, walk_length=50, 
                    num_walks=50, workers=1, temp_folder='')
# train node2vec model
n2w_model = node2vec.fit(window=5, min_count=1)
n2w_model.wv.save_word2vec_format('node_embv2')


Computing transition probabilities:  68%|██████▊   | 18947/27770 [00:18<00:03, 2440.99it/s]


Computing transition probabilities: 100%|██████████| 27770/27770 [00:20<00:00, 1369.57it/s]


In [96]:
from gensim.models import KeyedVectors
n2w_model = KeyedVectors.load_word2vec_format('node_emb', binary=False)

In [97]:
def get_node_emb(data) :
    node_emb = []
    for i, j in zip(data['node_1'], data['node_2']) :
        if i in G_data.nodes():
            emb_i = n2w_model.wv[str(i)]
        else :
            emb_i = np.zeros(128)
        if j in G_data.nodes():
            emb_j = n2w_model.wv[str(j)]
        else :
            emb_j = np.zeros(128)
        node_emb.append(np.hstack((emb_i, emb_j)))
        #node_emb.append(emb_i * emb_j)
    node_emb = np.asarray(node_emb)
    return node_emb #np.vstack((node_emb, node_emb_))
node_emb = get_node_emb(data_.drop(index = omissible_links_index))

In [98]:
node_emb.shape

(585512, 256)

In [106]:
#from sklearn.model_selection import GridSearchCV
#import sklearn
#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import StandardScaler
#xtrain, xtest, ytrain, ytest = train_test_split(node_emb, data['link'], 
#                                                test_size = 0.2, 
#                                                random_state = 35, stratify = data.link)

In [107]:
#print(xtrain.shape, xtest.shape)

In [401]:
## neural network

In [99]:
## Validation sur les noeuds omis !!
omitted_links = data_[data_.link == 1].iloc[omissible_links_index]
omitted_emb = get_node_emb(omitted_links)

In [100]:
y_train = data_.drop(index = omissible_links_index).link.values
omitted_emb.shape

(30000, 256)

In [101]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional, Input, Concatenate
from tensorflow.core.protobuf import rewriter_config_pb2
import tensorflow.keras.backend as K

In [74]:
tf.keras.backend.clear_session()  # For easy reset of notebook state.
model = Sequential()
#model.add(LSTM(32, dropout=0.1, recurrent_dropout=0.1, input_shape=(2, 128), activation='relu'))
#model.add(Bidirectional(LSTM(32, dropout=0.1,
#                             recurrent_activation='relu'), 
#                        input_shape=(2, 128)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=[f1_m])
bs = 1024
n_epochs = 10
history = model.fit(node_emb, y_train,
                    batch_size=bs, epochs=n_epochs,
                    validation_data=(omitted_emb, np.ones(len(omitted_emb))))
y_pred = model.predict(omitted_emb)
y_pred = np.where(y_pred >0.5, 1, 0)
print(sklearn.metrics.f1_score(np.ones(len(omitted_emb)), y_pred))

Train on 585512 samples, validate on 30000 samples
Epoch 1/10
585512/585512 [==============================] - 3s 5us/sample - loss: 0.2479 - f1_m: 0.9029 - val_loss: 0.1379 - val_f1_m: 0.9741
Epoch 2/10
585512/585512 [==============================] - 2s 4us/sample - loss: 0.1238 - f1_m: 0.9598 - val_loss: 0.1387 - val_f1_m: 0.9757
Epoch 3/10
585512/585512 [==============================] - 2s 4us/sample - loss: 0.1037 - f1_m: 0.9667 - val_loss: 0.1289 - val_f1_m: 0.9785
Epoch 4/10
585512/585512 [==============================] - 2s 4us/sample - loss: 0.0941 - f1_m: 0.9699 - val_loss: 0.1445 - val_f1_m: 0.9760
Epoch 5/10
585512/585512 [==============================] - 2s 4us/sample - loss: 0.0885 - f1_m: 0.9715 - val_loss: 0.1310 - val_f1_m: 0.9788
Epoch 6/10
585512/585512 [==============================] - 3s 4us/sample - loss: 0.0846 - f1_m: 0.9727 - val_loss: 0.1293 - val_f1_m: 0.9794
Epoch 7/10
585512/585512 [==============================] - 2s 4us/sample - loss: 0.0816 - f1_m: 

### Test concaténation des modèles

In [103]:
#Input image
input_emb = Input(shape=(256,))
input_attr = Input(shape=(15,))

encoded_1 = Dense(units=64, activation='relu')(input_emb)
encoded_1 = Dropout(0.1)(encoded_1)
encoded_2 = Dense(units=30, activation='relu')(input_attr)
encoded_2 = Dropout(0.1)(encoded_2)
encoded = Concatenate()([encoded_1, encoded_2])

output = Dense(units=16, activation='relu')(encoded)
output = Dropout(0.1)(output)
output = Dense(units=1, activation='sigmoid', name='autoencoder')(output)

autoencoder = Model(inputs = [input_emb, input_attr], outputs = output)
autoencoder.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics= [f1_m])

history = autoencoder.fit([node_emb, train], y_train,
                epochs=50,
                batch_size=50000,
                validation_data=([omitted_emb, val], y_val))

Train on 585512 samples, validate on 30000 samples
Epoch 1/50
585512/585512 [==============================] - 2s 4us/sample - loss: 0.7212 - f1_m: 0.5808 - val_loss: 0.3603 - val_f1_m: 0.9042
Epoch 2/50
585512/585512 [==============================] - 2s 4us/sample - loss: 0.5127 - f1_m: 0.7723 - val_loss: 0.3503 - val_f1_m: 0.9122
Epoch 3/50
585512/585512 [==============================] - 2s 4us/sample - loss: 0.4082 - f1_m: 0.8372 - val_loss: 0.3311 - val_f1_m: 0.9262
Epoch 4/50
585512/585512 [==============================] - 2s 4us/sample - loss: 0.3297 - f1_m: 0.8828 - val_loss: 0.2402 - val_f1_m: 0.9540
Epoch 5/50
585512/585512 [==============================] - 2s 3us/sample - loss: 0.2656 - f1_m: 0.9101 - val_loss: 0.1859 - val_f1_m: 0.9675
Epoch 6/50
585512/585512 [==============================] - 2s 3us/sample - loss: 0.2194 - f1_m: 0.9266 - val_loss: 0.1443 - val_f1_m: 0.9768
Epoch 7/50
585512/585512 [==============================] - 2s 3us/sample - loss: 0.1864 - f1_m: 

In [68]:
train.shape

(585512, 16)

In [181]:
omit = tmp.iloc[omissible_links_index]
omit [omit.diff_year < 0]

,pub_year_2,authors_x,node_2,pub_year_1,authors_y,node_1,link,diff_year
17,2000,[Paul S. Aspinwall],1001,1993,[C.N. Pope],9309125,0,-7
18,2000,[Paul S. Aspinwall],1001,1994,[A. de Rujula],9405191,0,-6
22,2000,[Paul S. Aspinwall],1001,1997,[Yoonbai Kim],9708038,0,-3
69,2000,"[M. Cvetic, H. Lu, C.N. Pope]",1002,1997,[],9701157,0,-3
90,2000,"[Y.S. Myung, Gungwon Kang]",1003,1992,"[H.Itoyama, B.Razzaghe-Ashrafi]",9204007,0,-8
...,...,...,...,...,...,...,...,...
55509,2000,"[Rabin Banerjee, Biswajit Chakraborty, Tomy ...",11011,1994,"[R.P. Manvelyan, R.L. Mkrtchyan (Yerevan Phys...",9401032,0,-6
55510,2000,"[Rabin Banerjee, Biswajit Chakraborty, Tomy ...",11011,1995,"[Ralf Kerschner (T""ubingen)]",9506055,0,-5
55511,2000,"[Rabin Banerjee, Biswajit Chakraborty, Tomy ...",11011,1996,"[Sergio Ferrara, Ramzi R. Khuri, Ruben Minas...",9602102,0,-4
55512,2000,"[Rabin Banerjee, Biswajit Chakraborty, Tomy ...",11011,1997,"[Sergio Ferrara, Murat Gunaydin]",9708025,0,-3


In [184]:
y_pred = autoencoder.predict([omitted_emb, val])[:, 0]
y_pred = np.where(y_pred >0.5, 1, 0)
print(sklearn.metrics.f1_score(np.ones(len(omitted_emb)), y_pred))

0.9934237015165749


## Prédiction sur les données test

In [104]:
data_test = pd.DataFrame({'node_1':[int(elt[0]) for elt in testing_set], 
                     'node_2':[int(elt[1]) for elt in testing_set]})
emb_test = get_node_emb(data_test)

In [105]:
pairs_test = [tuple(elt[0 : 2]) for elt in testing_set]
#y_train = [int(elt[-1]) for elt in training_set]

In [106]:
with Pool(4) as p: 
    features_test = p.map(get_features, pairs_test)
features_test = np.array(features_test)
test_tr = []
for elt in tqdm.tqdm(pairs_test) :
    test_tr.append(get_graph_feature(elt))
test_tr = np.asarray(test_tr)

100%|██████████| 32648/32648 [00:04<00:00, 7483.54it/s] 


In [107]:
## prédiction sur le test
test = np.hstack((test_tr, features_test))
test = scaler.transform(test)
print(train.shape, test.shape)
y_test = autoencoder.predict([emb_test, test])[:, 0]
y_test = np.where(y_test > 0.5, 1, 0)

(585512, 15) (32648, 15)


In [108]:
y_test.shape

(32648,)

In [ ]:
## re-entraînement sur tout le jeu de données
train = np.hstack((train_tr, features_nodes))
val = np.hstack((val_tr, features_val))
train = np.vstack((train, val))
train = scaler.fit_transform(train)

In [441]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional, Input
from tensorflow.core.protobuf import rewriter_config_pb2
import tensorflow.keras.backend as K
tf.keras.backend.clear_session()  # For easy reset of notebook state.
model = Sequential()
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(5, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=[f1_m])
bs = 2048
n_epochs = 30
history = model.fit(train, np.array(y_train),
                    batch_size=bs, epochs=n_epochs)
y_pred = model.predict(test)
#y_pred = np.where(y_pred >0.5, 1, 0)
#print(sklearn.metrics.f1_score(y_val, y_pred))

Epoch 1/30
615512/615512 [==============================] - 2s 3us/sample - loss: 0.6906 - f1_m: 0.6911
Epoch 2/30
615512/615512 [==============================] - 1s 1us/sample - loss: 0.6893 - f1_m: 0.7046
Epoch 3/30
615512/615512 [==============================] - 1s 1us/sample - loss: 0.6892 - f1_m: 0.7049
Epoch 4/30
615512/615512 [==============================] - 1s 1us/sample - loss: 0.6892 - f1_m: 0.7049
Epoch 5/30
615512/615512 [==============================] - 1s 1us/sample - loss: 0.6892 - f1_m: 0.7050
Epoch 6/30
615512/615512 [==============================] - 1s 1us/sample - loss: 0.6892 - f1_m: 0.7049
Epoch 7/30
615512/615512 [==============================] - 1s 1us/sample - loss: 0.6892 - f1_m: 0.7049
Epoch 8/30
615512/615512 [==============================] - 1s 1us/sample - loss: 0.6892 - f1_m: 0.7049
Epoch 9/30
615512/615512 [==============================] - 1s 2us/sample - loss: 0.6892 - f1_m: 0.7050
Epoch 10/30
615512/615512 [==============================] - 1s 

In [109]:
predictions_SVM = zip(range(len(testing_set)), y_test)
with open("improved_predictions.csv","w") as pred1:
    csv_out = csv.writer(pred1)
    csv_out.writerow(('id', 'category'))
    for row in predictions_SVM:
        csv_out.writerow(row)